# WikiNet

This is the python notebook file to train and run experiments from https://medium.com/@alexanderjhurtado/3f149676fbf3

The github with more files and information can be found at: https://github.com/alexanderjhurtado/cs224w_final

WikiNet is a model we have created to tackle the target prediction problem on the Wikispeedia dataset. Namely, given a sequence of articles clicked by a plae, our task is to predict the final target article the user is searching for. The following code is of our model definition, training, and evaluation for our experiments.

First, we begin by installing the necessary libraries!

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric

Here, we import all libraries that will be used by our code.

In [ ]:
import json
import pandas as pd
import time
import networkx as nx
from torch_geometric.utils import from_networkx

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCN, GAT, GraphSAGE
from torch.utils.data import Dataset, DataLoader

Download *graph_with_features.gml* and *paths_and_labels.tsv* from the GitHub repository. It can be found under the *colab_starter_pack/* folder. Upload these files to the current runtime to be able to run the following cell, which gets our fully preprocessed graph and path dataset. To see our data preprocessing code, look under the *extra_data_processing/* folder in our repository. 

In [ ]:
nx_graph = nx.read_gml('graph_with_features.gml')
G = from_networkx(nx_graph, group_node_attrs=['out_degree', 'in_degree', 'category_multi_hot', 'article_embed'])

path_data = pd.read_csv('paths_and_labels.tsv', sep='\t', header=None)

The following function will be called during training and evaluation to evaluate the model on the validation and test datasets.

In [ ]:
def get_evaluation_metrics(model, device, dataloader, dataset_size):
    model.eval()
    avg_loss = 0
    num_correct = 0
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            # get data
            inputs, labels = data['indices'].to(device), data['label'].to(device)
            outputs = model(inputs)
            # get loss
            loss = F.nll_loss(outputs, labels)
            avg_loss += loss.item()
            # get accuracy
            pred = outputs.argmax(dim=1)
            correct = (pred == labels).sum()
            num_correct += correct
    acc = int(num_correct) / dataset_size
    avg_loss /= dataset_size
    return acc, avg_loss

This defines the dataset class we use to represent our path data.

In [ ]:
class CustomPathDataset(Dataset):
    def __init__(self, path_data):
        self.x = path_data[0].apply(json.loads)
        self.labels = path_data[1]
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        x = torch.LongTensor(self.x[idx])
        label = self.labels[idx]
        sample = {"indices": x, "label": label}
        return sample

This is the class definition for our baseline model, an LSTM. Run this cell to be able to train the baseline model.

In [ ]:
class Baseline(torch.nn.Module):
    def __init__(self, graph, device, node_embed_size=64, lstm_hidden_size=32):
        super().__init__()
        self.graphX = graph.x.to(device)
        self.graphEdgeIndex = graph.edge_index.to(device)
        self.lstm_input_size = node_embed_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size,
                            hidden_size=lstm_hidden_size,
                            batch_first=True)
        self.pred_head = nn.Linear(lstm_hidden_size, self.graphX.shape[0])

    def forward(self, indices):
        node_emb = self.graphX
        node_emb_with_padding = torch.cat([node_emb, torch.zeros((1, self.lstm_input_size)).to(device)])
        paths = node_emb_with_padding[indices]
        _, (h_n, _) = self.lstm(paths)
        predictions = self.pred_head(torch.squeeze(h_n))
        return F.log_softmax(predictions, dim=1)

This is the class definition for our Graph Neural Network - based model. In our experiments, we ran three different GNNs: Graph Convolutional Networks (GCN), Graph Attention Networks (GAT), and GraphSAGE. Our GraphSAGE model performed the best and is the current model used in our class definition. If you would like to use GCN or GAT, simply replace `self.gnn = GraphSAGE(...)` with `self.gnn = GCN(...)` or `self.gnn = GAT(...)`, respectively. The arguments are the same for all 3 models.

This cell also defines our model weights file. This file will be generated during training, storing the weights for the best model based on validation accuracy during training.

In [ ]:
MODEL_WEIGHT_PATH = "model_weights.pth"

class Model(torch.nn.Module):
    def __init__(self, graph, device, sequence_path_length=32, gnn_hidden_size=128, node_embed_size=64, lstm_hidden_size=32):
        super().__init__()
        self.graphX = graph.x.to(device)
        self.graphEdgeIndex = graph.edge_index.to(device)
        self.gnn = GraphSAGE(in_channels=self.graphX.shape[1], 
                       hidden_channels=gnn_hidden_size, 
                       num_layers=3, 
                       out_channels=node_embed_size, 
                       dropout=0.1)
        self.batch_norm_lstm = nn.BatchNorm1d(sequence_path_length)
        self.batch_norm_linear = nn.BatchNorm1d(lstm_hidden_size)
        self.lstm_input_size = node_embed_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size,
                            hidden_size=lstm_hidden_size,
                            batch_first=True)
        self.pred_head = nn.Linear(lstm_hidden_size, self.graphX.shape[0])

    def forward(self, indices):
        node_emb = self.gnn(self.graphX, self.graphEdgeIndex)
        node_emb_with_padding = torch.cat([node_emb, torch.zeros((1, self.lstm_input_size)).to(device)])
        paths = node_emb_with_padding[indices]
        paths = self.batch_norm_lstm(paths)
        _, (h_n, _) = self.lstm(paths)
        h_n = self.batch_norm_linear(torch.squeeze(h_n))
        predictions = self.pred_head(h_n)
        return F.log_softmax(predictions, dim=1)

Here, we set up our `train / val / test` split as `90 / 5 / 5`. Moreover, we define our hyperparameters, including the learning rate, our optimizer (Adam), and the batch size.

To train on the baseline model instead of the GNN-based model, replace the line `model = Model(G, device).to(device)` to `model = Baseline(G, device).to(device)`.

In [ ]:
# set up our model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(G, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# get our dataset + splits
dataset = CustomPathDataset(path_data)
train_size = int(0.9 * len(dataset))
test_size = int(0.05 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# set up for training + validation
batch_size = 1024
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

This is our training script. We train our model for 200 epochs and print training loss, validation loss, validation accuracy, and time spent for each epoch.

Moreover, we train by running one batch through the model at a time and using the Negative Log Likelihood loss function. We also save the model weights for the best validation accuracy we see after an epoch. These weights will be used in the evaluation step.

In [ ]:
best_acc = 0
training_losses = []
validation_losses = []
validation_accs = []
model.train()
for epoch in range(200):  # loop over the dataset multiple times
    print('Epoch:', epoch+1)
    model.train()
    epoch_loss = 0
    start_time = time.time()
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['indices'].to(device), data['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = F.nll_loss(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    # validate epoch and print results
    training_losses.append(epoch_loss / train_size)
    print('Training Loss:', training_losses[-1])
    acc, valid_loss = get_evaluation_metrics(model, device, validloader, val_size)
    validation_losses.append(valid_loss)
    validation_accs.append(acc)
    if acc > best_acc:
        torch.save(model.state_dict(), MODEL_WEIGHT_PATH)
        best_acc = acc
    print("Validation accuracy:", acc)
    print("Validation loss:", valid_loss)
    print('Time elapsed:', time.time() - start_time)
    print()

This code runs evaluation on our test dataset. In particular, it uses the weights from the best validation accuracy to obtain our test accuracy. If you do not want to use those weights, you can comment out the line `model.load_state_dict(...)` to use the final weights from training instead.

This cell will print out the "loss" and accuracy on the testing dataset.

In [ ]:
model.load_state_dict(torch.load(MODEL_WEIGHT_PATH))
model.eval()
acc, test_loss = get_evaluation_metrics(model, device, testloader, test_size)
print("Test accuracy:", acc)
print("Test loss:", test_loss)